In [7]:
import pandas as pd
import my_utils as mu

In [8]:
df = mu.get_rawdata('GOLD')

get summonerId....


100%|██████████| 4/4 [00:02<00:00,  1.35it/s]


get puuId.....


100%|██████████| 12/12 [00:06<00:00,  1.87it/s]


get match_id....


100%|██████████| 12/12 [00:07<00:00,  1.65it/s]


get matches & timeline....


100%|██████████| 36/36 [00:50<00:00,  1.40s/it]


In [21]:
def lane_processing(tower_lane, tower_team, lane, team):
    if (tower_lane[0] == lane[0]) and (tower_team == team):
        return 1
    else:
        return 0
    

In [66]:
def assi_calc(y):
    try:
        return ','.join(list(map(lambda x: str(x), y['assistingParticipantIds'])))
    except:
        return ''

In [72]:
df_creates = []
for i in range(len(df)):
#     try:
        match = df.iloc[i].matches['info']['participants']
        for j in range(len(match)):
            match_id = df.iloc[i].match_id
            game_duration = df.iloc[i].matches['info']['gameDuration']
            game_version = df.iloc[i].matches['info']['gameVersion']
            try:
                riotIdGameName = match[j]['riotIdGameName']
            except:
                riotIdGameName = ''
            try:
                riotIdTagline = match[j]['riotIdTagline']
            except:
                riotIdTagline = ''
            summoner_level = match[j]['summonerLevel']
            participants_id = match[j]['participantId']
            champion_name = match[j]['championName']
            champ_experience = match[j]['champExperience']
            team_position = match[j]['teamPosition']
            team_id = match[j]['teamId']
            win = match[j]['win']
            kills = match[j]['kills']
            deaths = match[j]['deaths']
            assists = match[j]['assists']
            total_damage_dealt = match[j]['totalDamageDealtToChampions']
            total_damage_taken = match[j]['totalDamageTaken']


            df_list = list(map(lambda x: x['events'],df.iloc[i].timelines['info']['frames']))
            event_list = [element for array in df_list for element in array]
            tower_log = list(filter(lambda x:x['type'] == 'BUILDING_KILL' ,event_list))

            try:
                ft_tower_lane = tower_log[0]['laneType']
                ft_tower_team = tower_log[0]['teamId']
            except:
                ft_tower_lane = 'n'
                ft_tower_team = 'n'
            ft = lane_processing(ft_tower_lane,ft_tower_team,team_position,team_id)

            tower_tmp = list(map(lambda x: (x['laneType'],x['teamId'],x['timestamp']) ,tower_log))

            try:
                laneTower = list(filter(lambda x:(x[0][0] == team_position[0]) & (x[1] == team_id) ,tower_tmp))[0]
                lane_flag = 1
                laneTowerTime = laneTower[-1]
            except:
                lane_flag = 0
                laneTowerTime = 0
            
            blue_ban = list(map(lambda x: str(x['championId']) ,df.iloc[i].matches['info']['teams'][0]['bans']))
            red_ban = list(map(lambda x: str(x['championId']) ,df.iloc[i].matches['info']['teams'][1]['bans']))
            ban_list = '|'.join(list(set(blue_ban + red_ban)))
            
            
            kill_log = list(map(lambda x: list(filter(lambda z: z['type']=='CHAMPION_KILL',x['events'])),df.iloc[i].timelines['info']['frames']))
            kill_log = [element for array in kill_log for element in array]
            k = '|'.join(list(map(lambda x : str(x['killerId']) , kill_log)))
            v = '|'.join(list(map(lambda x :str(x['victimId']), kill_log)))
            a = '|'.join(list(map(lambda x : assi_calc(x) ,kill_log)))
            
            try:
                g15 = df.iloc[i].timelines['info']['frames'][15]['participantFrames'][(str(j+1))]['totalGold']
            except:
                g15 = 0
        
            df_creates.append([match_id,game_duration,game_version,riotIdGameName,riotIdTagline,summoner_level,participants_id,
                              champion_name,champ_experience,team_position,team_id,win,kills,deaths,assists,total_damage_dealt,total_damage_taken,
                              ft,lane_flag,laneTowerTime,ban_list,k,v,a,g15])
#     except:
#         continue
# columns =['matchId', 'gameDuration', 'gameVersion', 'riotIdGameName','riotIdTagline', 
#            'summonerLevel', 'participantId', 'championName', 'champExperience', 
#            'teamPosition', 'teamId', 'win', 'kills', 'deaths', 'assists', 'totalDamageDealtToChampions', 'totalDamageTaken',
#          'firstDT','laneTower','LaneTowerTime','bans','killerId','victimId','assistId','G15']
# df2 = pd.DataFrame(df_creates , columns = columns)

In [71]:
df2

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,totalDamageDealtToChampions,totalDamageTaken,firstDT,laneTower,LaneTowerTime,bans,killerId,victimId,assistId,G15
0,KR_6807387096,1430,13.22.541.9804,,,376,1,Illaoi,14114,TOP,...,29067,24784,0,0,0,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",6099
1,KR_6807387096,1430,13.22.541.9804,,,339,2,FiddleSticks,9808,JUNGLE,...,5728,20484,0,0,0,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",4640
2,KR_6807387096,1430,13.22.541.9804,,,174,3,Jayce,11558,MIDDLE,...,10837,9647,0,0,0,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",5582
3,KR_6807387096,1430,13.22.541.9804,,,686,4,Ezreal,12099,BOTTOM,...,30644,12606,0,1,1046702,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",7784
4,KR_6807387096,1430,13.22.541.9804,,,79,5,Ashe,6663,UTILITY,...,7104,14937,0,0,0,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",4472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,KR_6851691408,2167,13.24.547.9214,kirosha,KR1,123,6,Singed,19099,MIDDLE,...,24096,39046,0,1,897421,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",4549
356,KR_6851691408,2167,13.24.547.9214,왕자관,KR1,372,7,Vayne,14022,BOTTOM,...,28818,28857,0,1,1139045,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",5747
357,KR_6851691408,2167,13.24.547.9214,돈절래,KR157,213,8,Talon,14934,TOP,...,18071,29191,0,1,1650103,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",5231
358,KR_6851691408,2167,13.24.547.9214,부천 샹크스,KR1,336,9,LeeSin,16512,JUNGLE,...,25836,42045,0,0,0,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",6230


In [17]:
df_list = list(map(lambda x:x['events'] ,df.iloc[i].timelines['info']['frames']))
event_list = [element for array in df_list for element in array]
tower_log = list(filter(lambda x:x['type'] == 'BUILDING_KILL' ,event_list))

In [22]:
tower_log[0]

{'bounty': 0,
 'buildingType': 'TOWER_BUILDING',
 'killerId': 0,
 'laneType': 'TOP_LANE',
 'position': {'x': 981, 'y': 10441},
 'teamId': 100,
 'timestamp': 806644,
 'towerType': 'OUTER_TURRET',
 'type': 'BUILDING_KILL'}

In [27]:
tower_tmp = list(map(lambda x: (x['laneType'],x['teamId'],x['timestamp']) ,tower_log))

In [55]:
tower_tmp

[('TOP_LANE', 100, 806644),
 ('MID_LANE', 200, 897421),
 ('BOT_LANE', 200, 1139045),
 ('BOT_LANE', 100, 1193998),
 ('MID_LANE', 200, 1428983),
 ('TOP_LANE', 200, 1650103),
 ('BOT_LANE', 200, 1733057),
 ('MID_LANE', 200, 1809582),
 ('MID_LANE', 100, 1964146),
 ('BOT_LANE', 200, 1977831),
 ('MID_LANE', 100, 1987998),
 ('BOT_LANE', 200, 1990701),
 ('MID_LANE', 200, 2149406),
 ('MID_LANE', 200, 2155803),
 ('MID_LANE', 200, 2159363)]

In [ ]:
tower_tmp 

In [56]:
laneTower = list(filter(lambda x:(x[0][0] == team_position[0]) & (x[1] == team_id) ,tower_tmp))[0]

IndexError: list index out of range

In [57]:
[]

[]

In [60]:
blue_ban = list(map(lambda x: str(x['championId']) ,df.iloc[i].matches['info']['teams'][0]['bans']))
red_ban = list(map(lambda x: str(x['championId']) ,df.iloc[i].matches['info']['teams'][1]['bans']))
ban_list = '|'.join(list(set(blue_ban + red_ban)))

In [64]:
kill_log = list(map(lambda x: list(filter(lambda z: z['type']=='CHAMPION_KILL',x['events'])),df.iloc[i].timelines['info']['frames']))

In [65]:
kill_log

[[],
 [],
 [],
 [{'bounty': 400,
   'killStreakLength': 0,
   'killerId': 1,
   'position': {'x': 2351, 'y': 12231},
   'shutdownBounty': 0,
   'timestamp': 167253,
   'type': 'CHAMPION_KILL',
   'victimDamageDealt': [{'basic': False,
     'magicDamage': 0,
     'name': 'Talon',
     'participantId': 1,
     'physicalDamage': 150,
     'spellName': 'talonw',
     'spellSlot': 1,
     'trueDamage': 2,
     'type': 'OTHER'},
    {'basic': True,
     'magicDamage': 0,
     'name': 'Talon',
     'participantId': 1,
     'physicalDamage': 152,
     'spellName': 'talonq',
     'spellSlot': 64,
     'trueDamage': 0,
     'type': 'OTHER'},
    {'basic': False,
     'magicDamage': 0,
     'name': 'Talon',
     'participantId': 1,
     'physicalDamage': 122,
     'spellName': 'talonq',
     'spellSlot': 0,
     'trueDamage': 7,
     'type': 'OTHER'},
    {'basic': True,
     'magicDamage': 0,
     'name': 'Talon',
     'participantId': 1,
     'physicalDamage': 65,
     'spellName': 'talonbasica

In [80]:
blue_team = df2[df2.teamId == 100]
red_team = df2[df2.teamId == 200]

# blue_team이라는 데이터프레임 옆에 enemy_champ, enemy_G15의 컬럼 추가
# red_team 이라는 데이터 프레임 옆에 enemy_champ, enemy_G15의 컬럼 추가
# team_df = blue_team 밑에 concat 아래다가 red_team을 넣어주기

## lane_win 이라는 컬럼을 만들어서 g_15 > enemy_g15일 경우에는 1 반대일 경우에는 0을 입력 둘이 같을경우에도 0
## first_blood 라는 컬럼을 만들어서 killerId 컬럼에서 firstblood를 한 participants 번호일 경우 1 나머지 0
## kill_point라는 컬럼을 만들어서 (kills + assists /같은팀의 총 킬수)를 입력 (killerId를 이용하는방법 or kills를 이용하는 방법) 소수점 3자리

In [85]:
tmp = red_team[['matchId','gameVersion','teamPosition','championName','G15']].rename(columns = {'championName':'enemy_champ', 'G15':'enemy_G15'})

In [86]:
tmp

,matchId,gameVersion,teamPosition,enemy_champ,enemy_G15
5,KR_6807387096,13.22.541.9804,TOP,Ornn,4434
6,KR_6807387096,13.22.541.9804,JUNGLE,Gragas,4146
7,KR_6807387096,13.22.541.9804,MIDDLE,Yasuo,5464
8,KR_6807387096,13.22.541.9804,BOTTOM,Kaisa,4960
9,KR_6807387096,13.22.541.9804,UTILITY,Lux,3427
...,...,...,...,...,...
355,KR_6851691408,13.24.547.9214,MIDDLE,Singed,4549
356,KR_6851691408,13.24.547.9214,BOTTOM,Vayne,5747
357,KR_6851691408,13.24.547.9214,TOP,Talon,5231
358,KR_6851691408,13.24.547.9214,JUNGLE,LeeSin,6230


In [87]:
blue_team_merge = pd.merge(blue_team,tmp,on = ['matchId','gameVersion','teamPosition'])

In [89]:
blue_team_merge[blue_team_merge.matchId =='KR_6807387096']

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,firstDT,laneTower,LaneTowerTime,bans,killerId,victimId,assistId,G15,enemy_champ,enemy_G15
0,KR_6807387096,1430,13.22.541.9804,,,376,1,Illaoi,14114,TOP,...,0,0,0,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",6099,Ornn,4434
1,KR_6807387096,1430,13.22.541.9804,,,339,2,FiddleSticks,9808,JUNGLE,...,0,0,0,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",4640,Gragas,4146
2,KR_6807387096,1430,13.22.541.9804,,,174,3,Jayce,11558,MIDDLE,...,0,0,0,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",5582,Yasuo,5464
3,KR_6807387096,1430,13.22.541.9804,,,686,4,Ezreal,12099,BOTTOM,...,0,1,1046702,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",7784,Kaisa,4960
4,KR_6807387096,1430,13.22.541.9804,,,79,5,Ashe,6663,UTILITY,...,0,0,0,104|517|53|897|89|111|25|711|114|777,4|5|10|4|1|3|1|4|4|8|1|6|4|9|6|4|5|9|1|4|3|8|1...,10|9|5|7|6|8|6|9|10|3|6|1|10|5|1|9|10|5|6|9|10...,"5|4|7,9|||2||5|5||||3|7||5|4|7,10||2,3|4,5||||...",4472,Lux,3427


In [90]:
tmp_blue = blue_team[['matchId','gameVersion','teamPosition','championName','G15']].rename(columns = {'championName':'enemy_champ', 'G15':'enemy_G15'})

In [92]:
red_team_merge = pd.merge(red_team,tmp_blue,on = ['matchId','gameVersion','teamPosition'])

In [95]:
team_df = pd.concat([blue_team_merge,red_team_merge])

In [108]:
team_df[team_df.matchId =='KR_6851691408']

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,LaneTowerTime,bans,killerId,victimId,assistId,G15,enemy_champ,enemy_G15,lane_win,first_blood
175,KR_6851691408,2167,13.24.547.9214,실버정글똥그자체,KR1,419,1,Rumble,18850,TOP,...,806644,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",5329,Talon,5231,1,1
176,KR_6851691408,2167,13.24.547.9214,차은우뺨치는외모,KR1,335,2,Jax,21291,JUNGLE,...,0,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",5938,LeeSin,6230,0,0
177,KR_6851691408,2167,13.24.547.9214,캴 챔,KR1,120,3,Sylas,17786,MIDDLE,...,1964146,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",5098,Singed,4549,1,0
178,KR_6851691408,2167,13.24.547.9214,틀프트,KR1,441,4,Kaisa,16560,BOTTOM,...,1193998,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",5382,Vayne,5747,0,0
179,KR_6851691408,2167,13.24.547.9214,인형뽑는사람,KR1,49,5,Rakan,14815,UTILITY,...,0,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",3627,Thresh,3655,0,0
175,KR_6851691408,2167,13.24.547.9214,kirosha,KR1,123,6,Singed,19099,MIDDLE,...,897421,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",4549,Sylas,5098,0,0
176,KR_6851691408,2167,13.24.547.9214,왕자관,KR1,372,7,Vayne,14022,BOTTOM,...,1139045,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",5747,Kaisa,5382,1,0
177,KR_6851691408,2167,13.24.547.9214,돈절래,KR157,213,8,Talon,14934,TOP,...,1650103,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",5231,Rumble,5329,0,0
178,KR_6851691408,2167,13.24.547.9214,부천 샹크스,KR1,336,9,LeeSin,16512,JUNGLE,...,0,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",6230,Jax,5938,1,0
179,KR_6851691408,2167,13.24.547.9214,소환사이름을대시,KR1,220,10,Thresh,11631,UTILITY,...,0,555|84|53|-1|48|234|101|61|35,1|9|1|2|7|7|1|2|10|2|6|5|8|2|7|3|9|2|9|2|3|8|3...,8|1|9|8|5|4|8|9|5|10|3|7|1|10|5|7|3|6|2|9|10|5...,"||||9,10|6,9,10||3||3,4|7|2,4|9|5|10|5|6|3|6|1...",3655,Rakan,3627,1,0


In [99]:
team_df['lane_win'] = team_df.apply(lambda x:1 if  x.G15 > x.enemy_G15 else 0 ,axis=1)

In [105]:
team_df['first_blood'] = team_df.apply(lambda x:1 if str(x.participantId) == x.killerId.split('|')[0] else 0 ,axis=1)

In [104]:
team_df.iloc[0].killerId.split('|')[0]

'4'

# 킬포인트 공식 (kills + assists /같은팀의 총 킬수)

In [111]:
team_df.iloc[0].killerId.split('|')[0]

'4'